<img align="right" src="images/tf.png" width="128"/>
<img align="right" src="images/ninologo.png" width="128"/>
<img align="right" src="images/dans.png" width="128"/>

---

To get started: consult [start](start.ipynb)

---

# Part of Speech tagging

## Team

name | discipline | stage | affiliation
--- | --- | --- | ---
Alba de Ridder | Assyriology | master student | NINO, Leiden
Martijn Kokken | Assyriology | master student | NINO, Leiden
Dirk Roorda | Computer Science | researcher | DANS, Den Haag
Cale Johnson | Assyriology | researcher and lecturer | Univ Birmingham
Caroline Waerzeggers | Assyriology | director | NINO, Leiden

In [ ]:
COLOPHON = dict(
  acronym='ABB-pos',
  corpus='Old Babylonian Letter Corpus (ABB)',
  dataset='oldbabylonian',
  compiler='Dirk Roorda',
  editors='Alba de Ridder, Martijn Kokken',
  initiators='Cale Johnson, Caroline Waerzeggers',
  institute='NINO, DANS',
)

## Status

* 2019-06-06 Personal pronouns added
* 2019-06-05 Dirk has reorganised the messy code after the sprint into a repeatable and documented workflow.
  The workflow covers special cases, prepositions, and nouns, not yet the extra insights of the sprint.
* 2019-06-03/04 Martijn, Alba and Dirk do a two-day sprint to follow-up on heuristics supplied by Cale Johnson.
  Martijn and Alba provide extra insights.

# Introduction

We collect and execute ideas to tag all word occurrences with a part-of-speech, such as `noun`, `prep`, `verb`.

In the end, we intend to provide extra features to the Old Babylonian corpus, as a standard module that will be always loaded
alongside the corpus.

This notebook will produce some word-level features:

* `pos`: main category of the word: `noun`, `verb`, `prep`, `pcl` (particle)
* `subpos`: secondary category of the word: `rel` (relation), `neg` (negation)

But in the meanwhile, it is work in progress, and during the work we collect candidate assignments in sets, which we save to disk.

These sets correspond to `noun`, `prep`, `nonprep` words as far as we have tagged them in the current state of the workflow.

The sets are all saved in a file `sets.tfx`, both next to this notebook (so that you can get it through GitHub), as in a shared
Dropbox folder `obb`, so that the Akkadian specialists (Alba de Ridder, Martijn Kokken, Cale Johnson) have instant access to them and
can test them in their TF-browser.

See [pos](pos.ipynb) to see how you can make use of these results.

# Method

## Overview

We perform the following steps in that order:

### Known words
We identify a bunch of words in closed categories that tend to interfere with noun/verb detection.
After identification, we exclude them from all subsequent pattern detection.

### Prepositions
We detect a few prepositions, especially those that (nearly) always preceed a noun.

### Nouns
We use several markers to detect nouns:

* determinatives
* prepositions
* Sumerian logograms
* numerals

We collect the marked occurrences and then look up the unmarked occurrences of the same words.
In this way we extend the detection of nouns considerably.

We have to deal with one big complication, though: **unkowns**.
If we have marked word occurrences with unknown signs in them, we cannot be confident that unmarked occurrences
of the same thing are really occurrences of the same underlying word.

So, if we transfer categorizations from marked occurrences to unmarked occurrences, we only do so if
the word in question does not have unknowns.

We save a lot of intermediate sets: for each step we save the nouns that result from that step:

These sets may overlap.

We also save subsets of these sets, namely the occurrences that are positively marked, and
the occurrences that lack marking and have been inferred.
These marked and unmarked subsets of each step are disjoint. 

whole step | marked | unmarked
--- | --- | ---
`noundet` | `nounMdet` | `nounUdet`
`nounprep` | `nounMprep` | `nounUprep`
`nounlogo` | `nounMlogo` | `nounUlogo`
`nounnum` | `nounMnum` | `nounUnum`

**Note on determinatives**

Determinative and phonetic complements are signs marked in ATF by being inside `{ }`, and in TF by having `det=1`.
From now on, we will abbreviate it: a **det** is a determinative or a phonetic complement.

# Start the engines

We load the Python modules we need.

In [ ]:
import os
import collections

from tf.app import use

from pos import PosTag

We load the corpus and obtain a handle to it: `A`.

In [ ]:
A = use('oldbabylonian:local', checkout='local', hoist=globals(), silent='deep')

# Run the workflow

We set up the detection machinery.

In [ ]:
PT = PosTag(A)

## Step: Inventory

We collect all the words and their occurrences and sift through determinatives and numerals.

We make a dictionary of words and their occurrences.
When we compute the word form, we pick the basic info of a sign, not the full ATF-representation with flags and brackets.

We also store the form without the *dets* that are present in the word.

In [ ]:
PT.prepare()

## Step: Known words

The case specification is a string.

In [ ]:
cases = '''
  la + u2-ul + u2-la = pcl, neg
  sza = pcl, rel
  u3 + u2-lu + u2 = pcl, conj
  lu = pcl
  an-nu-um + an-ni-im + an-nu-u2 = prn, dem
  i-na-an-na + a-nu-um-ma = adv, tmp
'''

To be read as follows:

Each line specifies a bunch of words, separated by `+` on the left hand side of the `=`;
the right hand side specifies the categories those words receive, separated by `,`.

The first category is the `pos`, (main part-of-speech),
the second category is the `subpos` (sub category within the main part-of-speech).

We use abbreviated forms, because users of this dataset will have to type them quite often.

### Categories

category | subcategory | meaning
--- | --- | ---
`pcl` | &nbsp; | particle (unspecified)
`pcl` | `neg` | negative particle
`pcl` | `rel` | relative particle
`pcl` | `conj` | conjunction
`prn` | `dem` | demonstrative pronoun
`adv` | `tmp` | temporal adverb

In [ ]:
PT.doKnownCases(cases)

## Step: Pronouns - personal

In [ ]:
prnPrs = '''
nom:
  1csg:
    - a-na-ku
    - a-na-ku-ma
    - a-na-ku-u2
    - a-na-ku-u2-ma
    - a-na-ku-ma-mi
  2msg:
    - at-ta
    - at-ta-ma
    - at-ta-a
    - at-ta-a-ma
  2fsg:
    - at-ti
    - at-ti-ma
    - at-ti-i-ma
  3msg:
    - szu-u2
    - szu-u2-ma
  3fsg:
    - szi-i
    - szi-i-ma
  1mpl:
    - ni-nu
    - ni-i-ni
  2mpl:
    - at-tu-nu
    - at-tu-nu-ma
    - at-tu-nu-u2
    - at-tu-u2-nu
    - at-tu-u2-nu-ma
  2fpl:
    - at-ti-na-ma
  3mpl:
    - szu-nu
    - szu-nu-ma
    - szu-nu-mi
    - szu-nu-u2
  3fpl:
    - szi-na

obl:
  1csg:
    - ia-ti
    - ia-ti-i-ma
    - ia-a-ti
  2msg:
    - ka-ta
    - ka-ta-a-ma
    - ka-a-ti
    - ka-ti:
        - P510880 reverse:8
        - P306656 obverse:8
    - ka-ti-i:
        - P292855 obverse:4
        - P292983 obverse:4
  2fsg:
    - ka-ti
  3csg:
    - szu-a-ti
    - szu-a-tu
    - sza-a-ti
    - sza-a-tu
    - szi-a-ti
  1cpl:
    - ni-a-ti
  2mpl:
    - ku-nu-ti
  2fpl:
    - /
  3mpl:
    - szu-nu-ti
  3fpl:
    - szi-na-ti

dat:
  1csg:
    - ia-szi
    - ia-szi-im
    - ia-a-szi
    - ia-a-szi-im
  2csg:
    - ka-szi-im
    - ka-szi-im-ma
    - ka-a-szum
  3msg:
    - szu-a-szi-im
  1cpl:
    - /
  2mpl:
    - ku-nu-szi-im
  2fpl:
    - /
  3mpl:
    - szu-nu-szi-im-ma
  3fpl:
    - /
'''

In [ ]:
PT.doPrnPrs(prnPrs)

## Step: Prepositions

The following prepositions are known to precede nouns.

In [ ]:
preps = '''
  i-na
  a-na
  e-li
  isz-tu
  it-ti
  ar-ki
'''

In [ ]:
PT.doPreps(preps)

We have made a set of all non-prepositions, i.e. all word occurrences not of one of these prepositions.

## Step: Nouns

### pass: Determiners

We take all words that have a *det*.

We collect the *markedData* for this step: all words that have a *det* inside.

The *unmarkedData* for this step are the occurrences of the stripped forms of the marked words, i.e.
the forms with the *det*s removed.
But only if those forms do not have an unknown in them., i.e. a `x`, `n`, or `...`.

### pass: Prepositions

Words after the given set of prepositions are usually nouns.
However, sometimes there are multiple prepositions in a row.
We take care that we do not mark those second prepostions as nouns.

### pass: Sumerian logograms

Any word that has one or more Sumerian logograms in it, will be marked as noun.

Sumerian logograms are defined as signs within the scope of an enclosing `_ _` pair.

In TF such signs are characterized by having `langalt=1`.

The unmarked data are the occurrences of the same words, but where none of the signs have `langalt=1`.

### pass: Numerals

Numerals are individual signs, but they can be part of words.
In those cases, we call the whole word a numeral.

We consider the category of numeral words as a subcategory of the nouns.

Note that there are also unknown numerals: those with reading `n`.

A numeral is always marked, there is no concept of unmarked occurrences of numerals.

In [ ]:
PT.doNouns()

# Results

We specify the metadata that we want to include into our new features.

In [ ]:
metaData = {
  '': COLOPHON,
  'pos': {
    'valueType': 'str',
    'description': 'primary part-of-speech category on full words',
  },
  'subpos': {
    'valueType': 'str',
    'description': 'secondary category within part-of-speech on full words',
  },
  'cs': {
    'valueType': 'str',
    'description': 'grammatical case: nom, acc, acg, gen, dat',
  },
  'ps': {
    'valueType': 'str',
    'description': 'grammatical person: 1, 2, 3',
  },
  'gn': {
    'valueType': 'str',
    'description': 'grammatical gender: m, f, c',
  },
  'nu': {
    'valueType': 'str',
    'description': 'grammatical number: sg, du, pl',
  },
}

The next cell saves the features to disk, and the sets as well.

In [ ]:
PT.export(metaData)

---

next: [pos](pos.ipynb)

---

full posTag and pos notebooks on
[annotation/tutorials/oldbabylonian/cookbook](https://nbviewer.jupyter.org/github/annotation/tutorials/blob/master/oldbabylonian/cookbook)

full tutorial on
[annotation/tutorials/oldbabylonian](https://nbviewer.jupyter.org/github/annotation/tutorials/blob/master/oldbabylonian)